In [8]:
import pandas as pd
import random
import generate_classification_report as gcr
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

# Set path of CSVs here(can also be just one path, but need to edit this file)
PATH1 = "../Datasets/IMDB-Dataset-GoogleTranslate.csv"
PATH2 = "../Datasets/IMDB-Dataset-MideindTranslate.csv"

# Device and model to use
DEVICE = "cuda"
MODEL = "../Models/electra-base-google-batch8-remove-noise-model/"

# Columns to drop from CSVs (if applicable)
DROP_COLUMNS = ["num", "rating", "id", "movie", "rating"]

d1 = pd.read_csv(PATH1)
d2 = pd.read_csv(PATH2)


d1 = d1.sample(n=100)
d2 = d2.sample(n=100)



for col in DROP_COLUMNS:
    if col in d1.columns:
        d1.drop(col, axis=1, inplace=True)
    if col in d2.columns:
        d2.drop(col, axis=1, inplace=True)


df_orig = pd.merge(d1, d2, how="outer")
total = 0
for i in range(0, 10):
    r = random.randint(0, 10000)

    fifty_negative = (
        df_orig.where(lambda x: x["sentiment"] == "negative")
        .dropna()
        .sample(n=50, random_state=r)
    )
    fifty_positive = (
        df_orig.where(lambda x: x["sentiment"] == "positive")
        .dropna()
        .sample(n=50, random_state=r)
    )

    new_df = pd.merge(
        fifty_negative, fifty_positive, on=["sentiment", "review"], how="outer"
    )
    new_df.sentiment = new_df.sentiment.apply(lambda x: 1 if x == "positive" else 0)
    X_all = new_df.review
    y_all = new_df.sentiment
    accuracy = gcr.call_model(X_all, y_all, MODEL, DEVICE, accuracy=True)
    total += accuracy
    print("acc: {0:.4f}, seed: {1}, i: {2}".format(accuracy, r, i))


print("Average accuracy: ", total / 10)

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '../models/electra-base-google-batch8-remove-noise-model/'. Use `repo_type` argument if needed.